In [2]:
using DataFrames, CSV
using JuMP, GLPKMathProgInterface

In [3]:
function str2vec(str)
    S = split(str, " ");
    V = [parse(Int, s) for s in S];
end;

In [31]:
# Open schedule info
df_tensors = DataFrame(CSV.File("segmentation/tensor_details.csv"));
df_schedule = DataFrame(CSV.File("segmentation/execution_schedule_info.csv"));

fixed_tensors = [183, 168, 179, 166]

no_tensors = size(df_tensors, 1);
no_operators = size(df_schedule, 1);

df_tensors.Idx = 1:no_tensors;

MAX_MEM = 7 * 1024 * 1024;
MIN_CH = 32;
MAX_CH = 64 * 32;

In [32]:
opt_model = Model(solver=GLPKSolverMIP());

In [33]:
@variable(opt_model, x[1:no_tensors] >= MIN_CH, Int);

opt_model

Feasibility problem with:
 * 0 linear constraints
 * 71 variables: 71 integer
Solver is GLPKMathProgInterface.GLPKInterfaceMIP.GLPK

In [34]:
# Set max channels for other tensors
nfixed_df = df_tensors[.!in.(df_tensors.Id, Ref(fixed_tensors)), :];
for r in eachrow(nfixed_df)
    @constraint(opt_model, x[r.Idx] <= MAX_CH);
end

opt_model

Feasibility problem with:
 * 67 linear constraints
 * 71 variables: 71 integer
Solver is GLPKMathProgInterface.GLPKInterfaceMIP.GLPK

In [35]:
# Set fixed tensor constraints
fixed_df = df_tensors[in.(df_tensors.Id, Ref(fixed_tensors)), :]
for i in 1:length(fixed_tensors)
    idx = fixed_df[i, :].Idx;
    size = fixed_df[i, :].Size;
    
    @constraint(opt_model, x[idx] == size);
end
opt_model

Feasibility problem with:
 * 71 linear constraints
 * 71 variables: 71 integer
Solver is GLPKMathProgInterface.GLPKInterfaceMIP.GLPK

In [36]:
# Set constraints
for s in 1:no_operators
    tensors = str2vec(df_schedule[s, 2])
    sel = df_tensors[in.(df_tensors.Id, Ref(tensors)), :];
    c = 0;
    for r in eachrow(sel)
        if r.Id in fixed_tensors
            c = c + x[r.Idx];
        else
            shape = str2vec(r.Shape);
            feat = shape[1] * shape[2] * shape[3];
            c = c + feat * x[r.Idx];
        end
        
        if c != 0
            @constraint(opt_model, c <= MAX_MEM);
        end
    end
end

opt_model

Feasibility problem with:
 * 245 linear constraints
 * 71 variables: 71 integer
Solver is GLPKMathProgInterface.GLPKInterfaceMIP.GLPK

In [37]:
# Set cost function
@objective(opt_model, Max, sum(x));

opt_model

Maximization problem with:
 * 245 linear constraints
 * 71 variables: 71 integer
Solver is GLPKMathProgInterface.GLPKInterfaceMIP.GLPK

In [38]:
status = solve(opt_model)

:Optimal

In [39]:
print(getvalue(x))

[2048.0, 393.0, 48.0, 393.0, 1611.0, 32.0, 32.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 528.0, 528.0, 1177.0, 1177.0, 2048.0, 1209.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 91476.0, 5.548116e6, 2048.0, 2048.0, 2048.0, 2048.0, 91476.0, 792588.0]